<a href="https://colab.research.google.com/github/nutworker/qM-AI-L/blob/L_test/email-subject/model-tuning/AESLC_tuning_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
repository_url = 'https://github.com/nutworker/qM-AI-L'
!git clone {repository_url}

Cloning into 'qM-AI-L'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 111 (delta 45), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (111/111), 7.81 MiB | 3.25 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [2]:
!pip install transformers
!pip install evaluate
!pip install rouge-score # Installing rouge-score library (https://pypi.org/project/rouge-score/)

# !pip install accelerate -U

!pip install transformers[torch]
!pip install sacrebleu

!pip install datasets nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

In [3]:
# Importing Libraries


import os
import re
import pandas as pd # Data Handling
import tensorflow as tf
import numpy as np
import torch
import time
import seaborn as sns

from datasets import load_metric
from datasets import Dataset
import evaluate

from google.colab import files
import shutil

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Transformers
from transformers import BartTokenizer, BartForConditionalGeneration      # BERT Tokenizer and architecture
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments         # These will help us to fine-tune our model
from transformers import pipeline                                         # Pipeline
from transformers import DataCollatorForSeq2Seq                           # DataCollator to batch the data

from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
!pip install nltk
import nltk
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score

# **Load Dataset**

In [6]:
dataset_df = pd.read_csv('/content/qM-AI-L/email-subject/model-tuning/train_emails.csv')

In [7]:
dataset_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
0,allen-p_inbox_20,"Greg/Phillip, Attached is the Grande Communic...",Service Agreement,NaN,NaN,NaN,65,2,gregphillip attached is the grande communicati...
1,allen-p_inbox_28,Phillip & Keith Attached is the first draw re...,Bishops Corner,NaN,NaN,NaN,145,2,phillip keith attached is the first draw reque...
2,allen-p_inbox_63,Your Internet Banking accounts are now setup a...,Internet Banking,NaN,NaN,NaN,250,2,your internet banking accounts are now setup a...
3,allen-p_inbox_64,To our IBS Customers that are still hanging in...,Internet Banking,NaN,NaN,NaN,458,2,to our ibs customers that are still hanging in...
4,allen-p_inbox_65,Phillip Good Morning!\nI hope you had a wonder...,SMEs for expert stories,NaN,NaN,NaN,68,4,phillip good morning i hope you had a wonderfu...
...,...,...,...,...,...,...,...,...,...
14431,zufferli-j_inbox_43,This email is acknowledgement from the Power P...,Power Pool,NaN,NaN,NaN,227,2,this email is acknowledgement from the power p...
14432,zufferli-j_inbox_44,This email is acknowledgement from the Power P...,Power Pool of Alberta,NaN,NaN,NaN,277,4,this email is acknowledgement from the power p...
14433,zufferli-j_inbox_46,"John, Further to the voice message that I lef...",Enron Security,NaN,NaN,NaN,148,2,john further to the voice message that i left ...
14434,zufferli-j_inbox_8,Make sure that all curves are downloaded by th...,Simulation Curves,NaN,NaN,NaN,66,2,make sure that all curves are downloaded by th...


# **Test Train Split**

In [8]:
subset_df = dataset_df.sample(frac=0.4, random_state=42)
subset_df.shape

(5774, 9)

In [9]:
Train_df, Test_df = train_test_split(subset_df, test_size=0.2, random_state=42)

In [10]:
Train_df.shape, Test_df.shape

((4619, 9), (1155, 9))

In [11]:
Train_dataset = Dataset.from_pandas(Train_df)
Test_dataset = Dataset.from_pandas(Test_df)

In [12]:
Train_dataset, Test_dataset

(Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__'],
     num_rows: 4619
 }),
 Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__'],
     num_rows: 1155
 }))

**Create a subset of Dataset**

In [14]:
# #create a subset of dataset
# train_dataset = Train_dataset.filter(lambda example, index: index % 2 == 0, with_indices=True)
# test_dataset = Test_dataset.filter(lambda example, index: index % 2 == 0, with_indices=True)

Train_dataset.shape, Test_dataset.shape

((4619, 10), (1155, 10))

In [16]:
Train_dataset['cleaned_emails'][0]

'this follows up on my voicemail to you of just a few minutes ago. i reviewed a voicemail from theresa bushman this morning about the timing on the new mariner matter that mariner for which mariner would like to retain fj. she has spoken with scott josie apparently one of our business people and has been advised that the december 27 closing is critical due to market conditions. theresa also believes that the december 27 date will be aggressive even for fj given the late date. theresa please let me know if i have mischaracterized what you said. i see two options 1. if you agree with me that we cant sign the letter as is i do my best to get an acceptable letter negotiated with uriel dutton today. if i cant mariner retains someone else today. frankly i am not optimistic about getting an acceptable letter from fj based on where fj started. 2. if you believe that we need to go ahead and get the best deal we can from fj under the circumstances i will do my best to get that done by noon or so

**Load the pre-trained FLAN-T5 models and its tokenizer directly from HuggingFace. FLAN-T5 was released in the paper Scaling Instruction-Finetuned Language Models - it is an enhanced version of T5 that has been finetuned in a mixture of tasks. Setting torch_dtype=torch.bfloat16 specifies the memory type to be used by this model.**

In [17]:
# Function to get size of models
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"Trainable model parameters: {trainable_model_params}\nAll model parameters: {all_model_params}\nPercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



**Load Flan T5 Base Model**

In [36]:
# Load the model
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


# Freeze the first 6 layers of the encoder and decoder
for param in model.encoder.block[:6].parameters(): # Use 'block' instead of 'layers'
    param.requires_grad = False
for param in model.decoder.block[:6].parameters(): # Use 'block' instead of 'layers'
    param.requires_grad = False

print("For Model : FLAN-T5-Base \n",print_number_of_trainable_model_parameters(model))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


For Model : FLAN-T5-Base 
 Trainable model parameters: 148463616
All model parameters: 247577856
Percentage of trainable model parameters: 59.97%


In [29]:
print("For Model : FLAN-T5-Base \n",print_number_of_trainable_model_parameters(model))

For Model : FLAN-T5-Base 
 Trainable model parameters: 0
All model parameters: 247577856
Percentage of trainable model parameters: 0.00%


In [37]:
def tokenize_function(examples):
    inputs = ["generate subject line: " + example for example in examples["body"]] # Access the 'body' column
    targets = [example for example in examples["subject"]] # Access the 'subject' column
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [38]:
tokenized_train_dataset = Train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = Test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/4619 [00:00<?, ? examples/s]

Map:   0%|          | 0/1155 [00:00<?, ? examples/s]

In [58]:
tokenized_train_dataset, tokenized_eval_dataset

(Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 4619
 }),
 Dataset({
     features: ['source', 'body', 'subject', 'ann0', 'ann1', 'ann2', 'body_wcount', 'subj_wcount', 'cleaned_emails', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 1155
 }))

In [45]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",

    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),

)

trainer.train()

Epoch,Training Loss,Validation Loss
1,5.714500,0.152499
2,0.185400,0.139422
3,0.171000,0.137965


TrainOutput(global_step=1734, training_loss=1.7728679551385265, metrics={'train_runtime': 2252.6082, 'train_samples_per_second': 6.152, 'train_steps_per_second': 0.77, 'total_flos': 9488682632871936.0, 'train_loss': 1.7728679551385265, 'epoch': 3.0})

In [46]:
# Save the model
model_save_path = "fine_tuned_flan_t5_LP"
model.save_pretrained(model_save_path)

# Save the tokenizer
tokenizer.save_pretrained(model_save_path)

('fine_tuned_flan_t5_LP/tokenizer_config.json',
 'fine_tuned_flan_t5_LP/special_tokens_map.json',
 'fine_tuned_flan_t5_LP/spiece.model',
 'fine_tuned_flan_t5_LP/added_tokens.json')

In [48]:
# Load the model and tokenizer
Flan_T5_FT_LP_model = T5ForConditionalGeneration.from_pretrained(model_save_path)

Flan_T5_model_FT_tokenizer = T5Tokenizer.from_pretrained(model_save_path)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [55]:
emails_for_score_df = Test_df.sample(n=20, random_state=32)
emails_for_score_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
7222,mann-k_sent_2424,"Hi Chris, Happy New Year.\nHope all is going ...",Asset management,NaN,NaN,NaN,70,2,hi chris happy new year. hope all is going wel...
4359,hernandez-j_inbox_299,"To Our Valued Traffic Report Clients, Today...",Traffic Report outage and new product enhancem...,NaN,NaN,NaN,478,7,to our valued traffic report clients today fri...
5767,kaminski-v_sent_941,"Molly, This is an update on Anshuman.\nPlease...",Transition to Research Group - An Update,NaN,NaN,NaN,94,7,molly this is an update on anshuman. please se...
6817,lokey-t_inbox_85,Just A Reminder Jennifer Lev's Baby Shower W...,Reminder-Jennifer Lev's Baby Shower,NaN,NaN,NaN,64,4,just a reminder jennifer levs baby shower wedn...
1411,campbell-l_sent_73,Particpated in a Consolidated audit of two fac...,Weekly Report,NaN,NaN,NaN,110,2,particpated in a consolidated audit of two fac...
3608,germany-c_sent_452,We have a new Transco 6-6 contract.\nTerm 9/6...,New Trco capacity,NaN,NaN,NaN,34,3,we have a new transco 66 contract. term 96 930...
11633,shackleton-s_sent_6283,Marie: Shemeika Landry (X52552) and Shelli Sm...,procedure for opening brokerage account,NaN,NaN,NaN,31,5,marie shemeika landry x52552 and shelli smith ...
13868,whalley-g_inbox_414,"Headline: ""Unocal, Tesoro, Pioneer, Equiva, R...","Unocal, Tesoro, Pioneer, Equiva, Reliant to Pr...",NaN,NaN,NaN,718,11,headline unocal tesoro pioneer equiva reliant ...
327,beck-s_inbox_497,"Sally, As per our phone conversation, here is...",Sizing the Back-Office Outsourcing Opportunity,NaN,NaN,NaN,44,5,sally as per our phone conversation here is a ...
2492,delainey-d_sent_206,"Guys, I would like your opinion on this potent...",GE Plastics Facts,NaN,NaN,NaN,150,3,guys i would like your opinion on this potenti...


In [56]:
original_subjects = []
flan_T5_subjects = []
finetuned_flan_T5_subjects = []


for index, row in emails_for_score_df.iterrows():
    email = row['body']
    original_subject = row['subject']
    prompt = f"""
    Generate the subject line for the following email.

    Email:
    {email}

    Subject:
    """

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    Flan_T5_FT_LP_model.to(device)

    original_model_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    finetuned_model_input_ids = Flan_T5_model_FT_tokenizer(prompt, return_tensors="pt").input_ids.to(device)


    original_model_outputs = model.generate(input_ids=original_model_input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    flan_T5_subjects.append(original_model_text_output)

    instruct_model_outputs = Flan_T5_FT_LP_model.generate(input_ids=finetuned_model_input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = Flan_T5_model_FT_tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    finetuned_flan_T5_subjects.append(instruct_model_text_output)

    original_subjects.append(original_subject)

zipped_subjects = list(zip(original_subjects, flan_T5_subjects, finetuned_flan_T5_subjects))

Flan_T5_LP_df = pd.DataFrame(zipped_subjects, columns = ['original_subjects', 'T5_subjects', 'finetuned_flan_T5_subjects'])
Flan_T5_LP_df

,original_subjects,T5_subjects,finetuned_flan_T5_subjects
0,Asset management,Asset Management,Asset Management
1,Traffic Report outage and new product enhancem...,FriedWire Traffic Report,FriedWire Traffic Report
2,Transition to Research Group - An Update,Anshuman,Anshuman
3,Reminder-Jennifer Lev's Baby Shower,Jennifer Lev's Baby Shower,Jennifer Lev's Baby Shower
4,Weekly Report,Northern Border Pipeline Audit,Northern Border Pipeline Audit
5,New Trco capacity,Transco 6-6 Contract,Transco 6-6 Contract
6,procedure for opening brokerage account,Brokerage Account Procedures,Brokerage Account Procedures
7,"Unocal, Tesoro, Pioneer, Equiva, Reliant to Pr...",Interactive Energy 2001 - Opening-Day Insights,Interactive Energy 2001 - Opening-Day Insights
8,Sizing the Back-Office Outsourcing Opportunity,Sizing the Opportunity,Sizing the Opportunity
9,GE Plastics Facts,GE/GE/GE/GE deal,GE/GE/GE/GE deal


In [57]:

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

finetuned_model_results = rouge.compute(
    predictions=finetuned_flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

print('FLAN T5 Base Model:')
print(original_model_results)
print('\nFINETUNED FLAN T5 BASE MODEL:')
print(finetuned_model_results)

FLAN T5 Base Model:
{'rouge1': 0.4021085858585859, 'rouge2': 0.17678571428571427, 'rougeL': 0.39109848484848486, 'rougeLsum': 0.3942424242424243}

FINETUNED FLAN T5 BASE MODEL:
{'rouge1': 0.4021085858585859, 'rouge2': 0.17678571428571427, 'rougeL': 0.39109848484848486, 'rougeLsum': 0.3942424242424243}


**flan_t5_base_original_model**

In [61]:
flan_t5_base_original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)
flan_t5_base_tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

print("For Model : FLAN-T5-Base \n",print_number_of_trainable_model_parameters(flan_t5_base_original_model))

For Model : FLAN-T5-Base 
 Trainable model parameters: 247577856
All model parameters: 247577856
Percentage of trainable model parameters: 100.00%


In [ ]:
flan_t5_base_tokenizer.special_tokens_map

**Load Flan T5 Small Model**

In [60]:
flan_t5_small_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", torch_dtype=torch.bfloat16)
# flan_t5_small_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

print("For Model : Flan-t5-small \n",print_number_of_trainable_model_parameters(flan_t5_small_model))

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

For Model : Flan-t5-small 
 Trainable model parameters: 76961152
All model parameters: 76961152
Percentage of trainable model parameters: 100.00%


In [62]:
# flan-t5-base vs. flan-t5-small parameters
247577856-76961152

170616704

# **Test the Model with Zero Shot Inferencing**

**Using a pre-trained model from Hugging Face/sources - FLAN-T5-Base .**

In [63]:
index = 9

email = Train_dataset['cleaned_emails'][index]
subject = Train_dataset['subject'][index]

prompt = f"""
Generate the subject line for the following email.

Email:
{email}

Subject:
"""

inputs = flan_t5_base_tokenizer(prompt, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(flan_t5_base_original_model.device) for k, v in inputs.items()}


output = flan_t5_base_tokenizer.decode(
    flan_t5_base_original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FLAN-T5 MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Generate the subject line for the following email.

Email:
edmund i have reviewed the long descriptions i was given for the credit derivatives i cant tell if they are current and have substantial comments i have attached what i have come up with sorry its in 2 pieces. basically i dont think we should be restating something in the long description that is already covered sometimes with different language in the gtc this only creates an ambiguity and is not necessary to form a complete contract. my other concern and i have not been able to draft language to fix it is that i cant find any definition of the seller payment amount and i dont think the buyer payment language works can you give me a call so we can discuss my concerns

Subject:

---------------------------------------------------------------------------------------------------
BASELINE ORIGINAL HUMAN ANNOTED SUBJEC

**Using a pre-trained model from Hugging Face etc- FLAN-T5-SMALL .**

In [64]:
inputs = flan_t5_base_tokenizer(prompt, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# inputs = {k: v.to(device) for k, v in inputs.items()}


output = flan_t5_base_tokenizer.decode(
    flan_t5_small_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FLAN-T5-SMALL MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Generate the subject line for the following email.

Email:
edmund i have reviewed the long descriptions i was given for the credit derivatives i cant tell if they are current and have substantial comments i have attached what i have come up with sorry its in 2 pieces. basically i dont think we should be restating something in the long description that is already covered sometimes with different language in the gtc this only creates an ambiguity and is not necessary to form a complete contract. my other concern and i have not been able to draft language to fix it is that i cant find any definition of the seller payment amount and i dont think the buyer payment language works can you give me a call so we can discuss my concerns

Subject:

---------------------------------------------------------------------------------------------------
BASELINE ORIGINAL HUMAN ANNOTED SUBJEC

**By testing with various models with the zero shot inferencing, we can see that the model struggles to extract the same subject line compared to the baseline subject, but it does pull out some important information from the email which indicates the models can be fine-tuned to the task at hand.**

# **Performing Full Fine-Tuning using FLAN-T5 BASE MODEL**

In [67]:
# # tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

# # # Define special tokens
# # special_tokens = {
# #     "additional_special_tokens": ["<EMAIL>", "</EMAIL>"]
# # }

# # # Add special tokens to the tokenizer
# # tokenizer.add_special_tokens(special_tokens)


# # Load the model
# model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')

# Resize the model's embeddings to accommodate the new tokens
flan_t5_base_original_model.resize_token_embeddings(len(flan_t5_base_tokenizer))



def tokenize_function(examples):
    start_prompt = 'Generate subject line for the email.\n\nEmail:'
    end_prompt = '\nSubject:\n'
    inputs = [start_prompt + email + end_prompt for email in examples['cleaned_emails']]
    model_inputs = flan_t5_base_tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    labels = flan_t5_base_tokenizer(examples['subject'], max_length=15, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = Train_dataset.map(tokenize_function, batched=True, remove_columns=Train_dataset.column_names)


Map:   0%|          | 0/4619 [00:00<?, ? examples/s]

In [71]:
full_dataset = Dataset.from_pandas(dataset_df)

In [75]:
tokenized_full_dataset = full_dataset.map(tokenize_function, batched=True, remove_columns=full_dataset.column_names)

Map:   0%|          | 0/14436 [00:00<?, ? examples/s]

In [76]:
tokenized_full_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 14436
})

In [79]:
# Split the dataset into train and test
split_dataset = tokenized_full_dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [80]:
train_dataset, test_dataset

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 11548
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 2888
 }))

In [87]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=0.0001,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

In [88]:
trainer = Trainer(
    model=flan_t5_base_original_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=flan_t5_base_tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)

)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.644100,1.244680


Epoch,Training Loss,Validation Loss
1,1.644100,1.244680
2,1.912500,1.232297
3,1.766400,1.231620


TrainOutput(global_step=8661, training_loss=2.3930341022254935, metrics={'train_runtime': 6701.21, 'train_samples_per_second': 5.17, 'train_steps_per_second': 1.292, 'total_flos': 2.372044512559104e+16, 'train_loss': 2.3930341022254935, 'epoch': 3.0})

In [89]:
# Save the fine-tuned model and tokenizer
save_directory = "./finetuned_flan_t5_base"
flan_t5_base_original_model.save_pretrained(save_directory)
flan_t5_base_tokenizer.save_pretrained(save_directory)

# Zip the saved model files
zip_filename = "finetuned_flan_t5_base.zip"
shutil.make_archive(base_name="finetuned_flan_t5_base", format="zip", root_dir=save_directory)

'/content/finetuned_flan_t5_base.zip'

In [90]:
#Download the zipped checkpoint
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# # Unzip model checkpoint as it is zipped
# # !unzip finetuned_flan_t5_base.zip -d ./finetuned_flan_t5_base

# # Load the tokenizer
# tokenizer = T5Tokenizer.from_pretrained('t5-base', legacy=False)

# # Add special tokens to the tokenizer
# num_added_tokens = tokenizer.add_special_tokens(special_tokens_map)


# model_path = "./finetuned_flan_t5_base"
# finetuned_tokenizer = T5Tokenizer.from_pretrained(model_path,  legacy=False)
# finetuned_model = T5ForConditionalGeneration.from_pretrained(model_path)

In [91]:
model_path = "./finetuned_flan_t5_base"
finetuned_tokenizer = T5Tokenizer.from_pretrained(model_path,  legacy=False)
finetuned_model = T5ForConditionalGeneration.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [92]:
print("For Finetuned Model : Flan-t5 \n",print_number_of_trainable_model_parameters(finetuned_model))

For Finetuned Model : Flan-t5 
 Trainable model parameters: 247534848
All model parameters: 247534848
Percentage of trainable model parameters: 100.00%


In [ ]:
import json

# Load special tokens from JSON file
with open('/content/finetuned_flan_t5_base/special_tokens_map.json', 'r') as file:
    special_tokens_map = json.load(file)

# Print special tokens to verify
print(special_tokens_map)

# **Evaluate the Flan T5-Base Model Quantitatively (with ROUGE Metric)**
The ROUGE metric helps quantify the validity of subject lines produced by models. It compares subjects to a "Annoted baseline" subject which is usually created by a human. While not perfect, it does indicate the overall increase in subject line generatiion effectiveness that we have accomplished by fine-tuning.

**Load the Saved Finetuned Model and Tokenizer**

In [94]:
emails_for_score_df = Train_df.sample(n=10, random_state=32)
emails_for_score_df

,source,body,subject,ann0,ann1,ann2,body_wcount,subj_wcount,cleaned_emails
12278,smith-m_inbox_340,Here is the information that you guys had aske...,Western Coal,NaN,NaN,NaN,51,2,here is the information that you guys had aske...
6194,lay-k_inbox_15,Dr. Kenneth L. Lay Chairman of the Board Enron...,"Three proposals, following up our recent meeting.",NaN,NaN,NaN,1325,7,dr. kenneth l. lay chairman of the board enron...
3399,germany-c_sent_1052,This deal has been entered a CNG South Citygat...,"Deal 218918, McIntosh Partnerships",NaN,NaN,NaN,48,4,this deal has been entered a cng south citygat...
7541,mann-k_sent_880,"Matt, Since so much of this related to tax, I ...",Enron - Livingston County PRIVILEGED AND CONFI...,NaN,NaN,NaN,68,7,matt since so much of this related to tax i th...
939,buy-r_inbox_497,"As you may have become aware, the company rece...",Securities Litigation,NaN,NaN,NaN,128,2,as you may have become aware the company recei...
401,beck-s_inbox_746,"Dear Nancy, Sally & Janet - Did you have a wo...",December Newsletter,NaN,NaN,NaN,183,2,dear nancy sally janet did you have a wonderfu...
1224,campbell-l_inbox_474,Market Participants: Attached below are the v...,Version Mapping for January 2001 and May 2001 ...,NaN,NaN,NaN,65,10,market participants attached below are the ver...
10307,sanchez-m_inbox_62,20K and Relay Benefiting Assist the Officer Oc...,LAST CALL: RACE REGISTRATION - 20K and Relay,NaN,NaN,NaN,130,8,20k and relay benefiting assist the officer oc...
8625,nemec-g_sent_2163,"Mark, I talked with Jeff Stone this morning c...",Station 13 Petition Status,NaN,NaN,NaN,143,4,mark i talked with jeff stone this morning con...
13145,taylor-m_inbox_299,"In order to complete our files, please send co...",Swapco -- Outstanding Items,NaN,NaN,NaN,241,4,in order to complete our files please send cop...


In [95]:
emails = emails_for_score_df['body']
original_subjects = emails_for_score_df['subject']
original_subjects


12278                                         Western Coal
6194     Three proposals, following up our recent meeting.
3399                    Deal 218918, McIntosh Partnerships
7541     Enron - Livingston County PRIVILEGED AND CONFI...
939                                  Securities Litigation
401                                    December Newsletter
1224     Version Mapping for January 2001 and May 2001 ...
10307        LAST CALL:  RACE REGISTRATION - 20K and Relay
8625                            Station 13 Petition Status
13145                          Swapco -- Outstanding Items
Name: subject, dtype: object

In [96]:
test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2888
})

In [97]:
original_subjects = []
flan_T5_subjects = []
finetuned_flan_T5_subjects = []


for index, row in emails_for_score_df.iterrows():
    email = row['body']
    original_subject = row['subject']
    prompt = f"""
    Generate the subject line for the following email.

    Email:
    {email}

    Subject:
    """

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    flan_t5_base_original_model.to(device)
    finetuned_model.to(device)

    original_model_input_ids = flan_t5_base_tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    finetuned_model_input_ids = finetuned_tokenizer(prompt, return_tensors="pt").input_ids.to(device)


    original_model_outputs = flan_t5_base_original_model.generate(input_ids=original_model_input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = flan_t5_base_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    flan_T5_subjects.append(original_model_text_output)

    instruct_model_outputs = finetuned_model.generate(input_ids=finetuned_model_input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = finetuned_tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    finetuned_flan_T5_subjects.append(instruct_model_text_output)

    original_subjects.append(original_subject)

zipped_subjects = list(zip(original_subjects, flan_T5_subjects, finetuned_flan_T5_subjects))

Flan_T5_df = pd.DataFrame(zipped_subjects, columns = ['original_subjects', 'T5_subjects', 'finetuned_flan_T5_subjects'])
Flan_T5_df

Token indices sequence length is longer than the specified maximum sequence length for this model (1839 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1838 > 512). Running this sequence through the model will result in indexing errors


,original_subjects,T5_subjects,finetuned_flan_T5_subjects
0,Western Coal,WSCC coal,WSCC coal
1,"Three proposals, following up our recent meeting.",OECD Forum 2001,OECD Forum 2001
2,"Deal 218918, McIntosh Partnerships",CNG South Citygate,CNG South Citygate
3,Enron - Livingston County PRIVILEGED AND CONFI...,Tax Presentation,Tax Presentation
4,Securities Litigation,Securities Litigation,Securities Litigation
5,December Newsletter,December Newsletter,December Newsletter
6,Version Mapping for January 2001 and May 2001 ...,January 2001 Version Mapping,January 2001 Version Mapping
7,LAST CALL: RACE REGISTRATION - 20K and Relay,20K and Relay - Benefiting Assist Officer,20K and Relay Benefiting Assist Officer
8,Station 13 Petition Status,WFEC Petition,WFEC Petition
9,Swapco -- Outstanding Items,Signatures for ERMT LLC,Signatures for ERMT LLC


In [98]:

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

finetuned_model_results = rouge.compute(
    predictions=finetuned_flan_T5_subjects,
    references=original_subjects,
    use_aggregator=True,
    use_stemmer=True,
)

print('FLAN T5 Base Model:')
print(original_model_results)
print('\nFINETUNED FLAN T5 BASE MODEL:')
print(finetuned_model_results)

FLAN T5 Base Model:
{'rouge1': 0.38186813186813184, 'rouge2': 0.26666666666666666, 'rougeL': 0.36190476190476184, 'rougeLsum': 0.35805860805860806}

FINETUNED FLAN T5 BASE MODEL:
{'rouge1': 0.38186813186813184, 'rouge2': 0.26666666666666666, 'rougeL': 0.36190476190476184, 'rougeLsum': 0.35805860805860806}


**The Finetuning results should show  improvement in all ROUGE metrics:**



In [99]:
print("Absolute percentage improvement of FINETUNED MODEL over HUMAN BASELINE")

improvement = (np.array(list(finetuned_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(finetuned_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of FINETUNED MODEL over HUMAN BASELINE
rouge1: 0.00%
rouge2: 0.00%
rougeL: 0.00%
rougeLsum: 0.00%


# **Perform Parameter Efficient Fine-Tuning (PEFT)**
Parameter Efficient Fine-Tuning (PEFT), which is more efficient than full fine-tuning and yields comparable results. PEFT, often referring to Low-Rank Adaptation (LoRA), enables fine-tuning with fewer compute resources, often a single GPU.

LoRA produces a small adapter (a few MBs) while keeping the original LLM unchanged. During inference, this adapter is combined with the original LLM, allowing multiple adapters to reuse the same LLM and reducing memory requirements for various tasks.

**Setup the PEFT/LoRA model for Fine-Tuning**

In [100]:
!pip install \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00


In [101]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [102]:
peft_model = get_peft_model(flan_t5_base_original_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

Trainable model parameters: 3538944
All model parameters: 251073792
Percentage of trainable model parameters: 1.41%


In [103]:
train_dataset, test_dataset

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 11548
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 2888
 }))

In [104]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11548
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2888
    })
})

In [105]:
# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

# # Preprocess function
# def preprocess_function(examples):
#     inputs = tokenizer(examples['body'], max_length=512, truncation=True, padding="max_length")
#     targets = tokenizer(examples['lable'], max_length=128, truncation=True, padding="max_length")
#     return {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask'], 'labels': targets['input_ids']} # Added attention_mask to the output

# # Apply preprocessing
# train_dataset = split_dataset['train'].map(preprocess_function, batched=True)
# val_dataset = split_dataset['validation'].map(preprocess_function, batched=True)

# Load base model
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [106]:
!nvidia-smi

Fri Jul 26 14:56:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0              27W /  70W |   8251MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# kill -9 <pid>

In [107]:
output_dir = f'./peft-subjectline-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1,
    remove_unused_columns=False
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

max_steps is given, it will override any value given in num_train_epochs


In [108]:
test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2888
})

In [109]:
peft_trainer.train()

Step,Training Loss


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'base_model.model.encoder.embed_tokens.weight', 'base_model.model.shared.weight', 'base_model.model.decoder.embed_tokens.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

In [ ]:
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    gradient_accumulation_steps=8,  # Accumulate gradients over multiple steps
    logging_dir='./logs',
    logging_steps=200,
    fp16=True,  # Enable mixed precision training
    remove_unused_columns=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


In [ ]:

# Save the model and tokenizer
model.save_pretrained('path/to/save/finetuned_model')
tokenizer.save_pretrained('path/to/save/finetuned_model')

In [ ]:
output_dir = f'./peft-subjectline-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    # learning_rate=1e-4, # Higher learning rate than full fine-tuning.
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # logging_steps=10,
    # max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=split_dataset['test'],
)


peft_trainer.train()



In [ ]:
peft_model_path="./peft-subjectline-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-dialogue-summary-checkpoint-from-s3/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [ ]:
!pip install nltk

import nltk
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize

# Download the 'punkt' resource
nltk.download('punkt')

nltk.download('wordnet') # Download the WordNet corpus

In [ ]:

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([word_tokenize(true_subject)], word_tokenize(generated_subject))

    # SACREBLEU
    sacre_bleu = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu

Flan_T5_df['rougeL'], Flan_T5_df['meteor'], Flan_T5_df['sacrebleu'] = zip(*Flan_T5_df.apply(
    lambda row: evaluate_metrics(row['original_subjects'], row['T5_subjects']), axis=1))

# Calculate average scores
average_rougeL = Flan_T5_df['rougeL'].mean()
average_meteor = Flan_T5_df['meteor'].mean()
average_sacrebleu = Flan_T5_df['sacrebleu'].mean()

print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')

In [ ]:
#T5 Flan Base
# Load the ROUGE metric
rouge_metric = load_metric('rouge')

def calculate_rouge_scores(predictions, references):
    rouge_metric.add_batch(predictions=predictions, references=references)
    result = rouge_metric.compute()
    return {
        'rouge1': result['rouge1'].mid.fmeasure * 100,
        'rouge2': result['rouge2'].mid.fmeasure * 100,
        'rougeL': result['rougeL'].mid.fmeasure * 100
    }

# Generate predictions and calculate ROUGE scores
predictions = []
references = []

for example in test_dataset.shuffle(seed=42).select(range(20)):
    input_ids = example['input_ids']
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)  # Add batch dimension
    outputs = finetuned_model.generate(input_ids)
    pred = finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)
    ref = finetuned_tokenizer.decode(example['labels'], skip_special_tokens=True)

    predictions.append(pred)
    references.append(ref)

# Calculate and print ROUGE scores for each test sample
rouge_scores = []
for pred, ref in zip(predictions, references):
    score = calculate_rouge_scores([pred], [ref])
    rouge_scores.append(score)
    print(f"Prediction: {pred}\nReference: {ref}\nROUGE Scores: {score}\n")

# Convert to DataFrame for better visualization
rouge_df = pd.DataFrame(rouge_scores)
rouge_df

In [ ]:
!pip install rouge-score
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [ ]:
small_test_df = test_df.sample(n=10, random_state=42)

In [ ]:
small_test_df

In [ ]:
def generate_subject_line(email_body):
    inputs = finetuned_tokenizer("Generate a subject line for the following email.: " + email_body, return_tensors="pt", max_length=512, truncation=True)
    # Move inputs to the same device as the model
    inputs = {k: v.to(finetuned_model.device) for k, v in inputs.items()}
    # Access input_ids as a key in the dictionary
    outputs = finetuned_model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)
    return finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)



In [ ]:
small_test_df['generated_subject_line'] = small_test_df['cleaned_emails'].apply(generate_subject_line)
small_test_df

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)


# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # Tokenize the input for METEOR
    true_subject_tokens = nltk.word_tokenize(true_subject)
    generated_subject_tokens = nltk.word_tokenize(generated_subject)

    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([true_subject_tokens], generated_subject_tokens) # Pass tokenized input

    # SACREBLEU - Use import sacrebleu directly
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu_score # Return the score, not the module

small_test_df['rougeL'], small_test_df['meteor'], small_test_df['sacrebleu'] = zip(*small_test_df.apply(
    lambda row: evaluate_metrics(row['subject'], row['generated_subject_line']), axis=1))

In [ ]:
small_test_df

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

In [ ]:
start_prompt = 'Generate the subject line for the following email.\n\nEmail:'
end_prompt = '\nSubject:\n'
start_prompt + email + end_prompt
print(start_prompt)
print(email)
print(end_prompt)

In [ ]:
# Tokenization for FLAN-T5 model
def flan_t5_base_tokenize_function(example):
    start_prompt = 'Generate the subject line for the following email.\n\nEmail:'
    end_prompt = '\nSubject:\n'
    prompt = [start_prompt + email + end_prompt for email in example["body"]]
    example['input_ids'] = flan_t5_base_tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = flan_t5_base_tokenizer(example["subject"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_train_datasets = train_dataset.map(flan_t5_base_tokenize_function, batched=True)
tokenized_test_datasets = test_dataset.map(flan_t5_base_tokenize_function, batched=True)


In [ ]:
# tokenized_train_datasets = tokenized_train_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)
# tokenized_test_datasets = tokenized_test_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

In [ ]:
tokenized_train_datasets, tokenized_test_datasets

**FLAN-T5 BASE - Fine-Tune the Model with the Preprocessed Tokenised Dataset utiliing the built-in Hugging Face Trainer class. Then compare it with reference to the original model.**

flan_t5_base_original_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base', torch_dtype=torch.bfloat16)
flan_t5_base_tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')

In [ ]:


tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

def tokenize_function(examples):
    start_prompt = 'Generate the subject line for the following email.\n\nEmail:'
    end_prompt = '\nSubject:\n'
    inputs = [start_prompt + email + end_prompt for email in examples['email_body']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    labels = tokenizer(examples['subject'], max_length=50, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)


In [ ]:
# # Tokenization
# def preprocess_data(examples):
#     inputs = ["generate subject line: " + email for email in examples['cleaned_emails']]
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples['subject'], max_length=128, truncation=True, padding="max_length")

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# train_dataset = train_dataset.map(preprocess_data, batched=True)
# test_dataset = test_dataset.map(preprocess_data, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=flan_t5_base_original_model ,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_datasets,
)

# Train the model
trainer.train()



FLAN-T5 SMALL - Fine-Tune the Model with the Preprocessed Tokenised Dataset utiliing the built-in Hugging Face Trainer class. Then compare it with reference to the original model.

flan_t5_small_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", torch_dtype=torch.bfloat16)
flan_t5_small_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [ ]:
# Tokenization for FLAN-T5 SMALL model
def flan_t5_small_tokenize_function(example):
    start_prompt = 'Generate the subject line for the following email.\n\nEmail:'
    end_prompt = '\nSubject:\n'
    prompt = [start_prompt + email + end_prompt for email in example["cleaned_emails"]]
    example['input_ids'] = flan_t5_small_tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = flan_t5_small_tokenizer(example["subject"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_train_datasets = train_dataset.map(flan_t5_small_tokenize_function, batched=True)
tokenized_test_datasets = test_dataset.map(flan_t5_small_tokenize_function, batched=True)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=flan_t5_small_model,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_datasets,
)

# Train the model
trainer.train()


In [ ]:
# Save the model checkpoint
flan_t5_small_model.save_pretrained('./results/Flan_T5_small_model')
flan_t5_small_tokenizer.save_pretrained('./results/Flan_T5_small_model')

# Define and save the generation configuration
generation_config = GenerationConfig(
    early_stopping=True,
    num_beams=4,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)

generation_config.save_pretrained('./results/Flan_T5_small_model')

In [ ]:
AutoModelForSeq2SeqLM
AutoTokenizer

In [ ]:
# Load the trained Flan T5 Small fine tuned model
flan_T5_small_model_directory = '/content/results/Flan_T5_small_model'
flan_T5_small_tokenizer = AutoTokenizer.from_pretrained(flan_T5_small_model_directory)
flan_T5_small_model_finetuned = AutoModelForSeq2SeqLM.from_pretrained(flan_T5_small_model_directory)

In [ ]:
emails_for_score = tokenized_test_datasets[0:10]['cleaned_emails']
original_subjects = tokenized_test_datasets[0:10]['subject']

In [ ]:
flan_T5small_subjects = []
finetuned_flan_T5small_subjects = []

for _, cleaned_emails in enumerate(emails_for_score):
    prompt = f"""
    Generate the subject line for the following email.

    Email:
    {email}

    Subject:
    """
    # Tokenize the input and move to the same device as the model
    inputs = flan_t5_small_tokenizer(prompt, return_tensors='pt').to(flan_T5_small_model_finetuned.device) # Move inputs to the same device as the fine-tuned model

    # Add the decoder_start_token_id to the generation configuration
    generation_config = GenerationConfig(max_new_tokens=200,
        decoder_start_token_id=flan_T5_small_tokenizer.bos_token_id if flan_T5_small_tokenizer.bos_token_id is not None else flan_T5_small_tokenizer.eos_token_id
    )



    # Generate subject using the original model
    # Make sure flan_t5_small_model is on the same device as flan_T5_small_model_finetuned
    flan_t5_small_model = flan_t5_small_model.to(flan_T5_small_model_finetuned.device)
    original_model_outputs = flan_t5_small_model.generate(**inputs, generation_config=generation_config)
    original_model_subject_output = flan_t5_small_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    flan_T5small_subjects.append(original_model_subject_output)

    # Generate subject using the finetuned model
    trained_flan_t5_outputs = flan_T5_small_model_finetuned.generate(**inputs, generation_config=generation_config)
    instruct_model_text_output = flan_T5_small_tokenizer.decode(trained_flan_t5_outputs[0], skip_special_tokens=True)
    finetuned_flan_T5small_subjects.append(instruct_model_text_output)

zipped_summaries = list(zip(original_subjects, flan_T5small_subjects, finetuned_flan_T5small_subjects))

Flan_T5_Small_Subjects_df = pd.DataFrame(zipped_summaries, columns = ['Original_subjects', 'Flan_T5s_subjects', 'Finetuned_Flan_T5s_subjects'])
Flan_T5_Small_Subjects_df

In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
# from datasets import Dataset


# # Initialize tokenizer and model
# tokenizer = T5Tokenizer.from_pretrained('t5-small')
# T5_model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [ ]:
def tokenize_function(example):
    start_prompt = 'Generate the subject line for the following email.\n\n'
    end_prompt = '\n\Subject: '
    prompt = [start_prompt + email + end_prompt for email in example["cleaned_emails"]]
    # Tokenize the input_ids - do not flatten
    example['input_ids'] = [tokenizer(p, padding="max_length", truncation=True, return_tensors="pt").input_ids[0] for p in prompt]
    # Tokenize the labels - do not flatten
    example['labels'] = [tokenizer(s, padding="max_length", truncation=True, return_tensors="pt").input_ids[0] for s in example["subject"]]

    return example

In [ ]:
# output_dir = f'./email-Subject-training-{str(int(time.time()))}'

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     learning_rate=1e-5,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     logging_steps=30,
#     evaluation_strategy='epoch',
#     per_device_train_batch_size=4,  # Reduced batch size
#     # max_steps=1
# )

# # training_args = TrainingArguments(
# #     output_dir='./results',
# #     evaluation_strategy='epoch',
# #     learning_rate=1e-5,
# #     per_device_train_batch_size=4,
# #     per_device_eval_batch_size=4,
# #     num_train_epochs=3,
# #     weight_decay=0.01,
# # )

# trainer = Trainer(
#     model=flan_t5_base_original_model,
#     args=training_args,
#     train_dataset=tokenized_train_datasets,
#     eval_dataset=tokenized_test_datasets
# )

# trainer.train()

In [ ]:
# # Generate subject lines for the test set
# def generate_subject_line(email_body):
#     inputs = tokenizer("generate subject line: " + email_body, return_tensors="pt", max_length=512, truncation=True)
#     # Move inputs to the same device as the model
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
#     outputs = model.generate(inputs.input_ids, max_length=128, num_beams=4, early_stopping=True)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# test_df['generated_subject_line'] = test_df['cleaned_emails'].apply(generate_subject_line)

# Generate subject lines for the test set
def generate_subject_line(email_body):
    inputs = tokenizer("generate subject line: " + email_body, return_tensors="pt", max_length=512, truncation=True)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    # Access input_ids as a key in the dictionary
    outputs = model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_df['generated_subject_line'] = test_df['cleaned_emails'].apply(generate_subject_line)

# **t5-small Evaluation**

In [ ]:
!pip install nltk
import nltk
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [ ]:
# Download the 'punkt' resource
nltk.download('punkt')
nltk.download('wordnet') # Download WordNet

from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)


# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # Tokenize the input for METEOR
    true_subject_tokens = nltk.word_tokenize(true_subject)
    generated_subject_tokens = nltk.word_tokenize(generated_subject)

    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([true_subject_tokens], generated_subject_tokens) # Pass tokenized input

    # SACREBLEU - Use import sacrebleu directly
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu_score # Return the score, not the module

test_df['rougeL'], test_df['meteor'], test_df['sacrebleu'] = zip(*test_df.apply(
    lambda row: evaluate_metrics(row['subject'], row['generated_subject_line']), axis=1))

# Calculate average scores
average_rougeL = test_df['rougeL'].mean()
average_meteor = test_df['meteor'].mean()
average_sacrebleu = test_df['sacrebleu'].mean() # Now you should be able to calculate the mean

print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')

# **Pretrained('facebook/bart-base')**

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
# # Convert DataFrame to Dataset
# train_dataset = Dataset.from_pandas(train_df)
# test_dataset = Dataset.from_pandas(test_df)

# Initialize tokenizer and model
facebook_bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
facebook_bart_base_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
print("For Model : facebook_bart_base_model \n",print_number_of_trainable_model_parameters(facebook_bart_base_model))

In [ ]:
prompt = f"""
Generate the subject line for the following email.

Email:
{email}

Subject:
"""

inputs = facebook_bart_tokenizer(prompt, return_tensors='pt')

# Move input tensors to the same device as the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(facebook_bart_base_model.device) for k, v in inputs.items()}

outputs = facebook_bart_base_model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ORIGINAL HUMAN ANNOTED SUBJECT LINE :\n{subject}\n')
print(dash_line)
print(f'FB BART MODEL GENERATION - ZERO SHOT SUBJECT LINE:\n{output}')

In [ ]:

# Tokenization
def Tokenization_preprocess_data(examples):
    inputs = [prompt + email for email in examples['cleaned_emails']]
    model_inputs = facebook_bart_tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Use facebook_bart_tokenizer as the target tokenizer
    with facebook_bart_tokenizer.as_target_tokenizer():
        labels = facebook_bart_tokenizer(examples['subject'], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(Tokenization_preprocess_data, batched=True)
test_dataset = test_dataset.map(Tokenization_preprocess_data, batched=True)

# # Define training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )


training_args = TrainingArguments(
    output_dir='./results',        # Directory to save the model checkpoints
    evaluation_strategy='steps',
    save_steps=500,               # Save checkpoint every 1000 steps
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=200,
)

# Define Trainer
facebook_bart_base_model_trainer = Trainer(
    model=facebook_bart_base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
facebook_bart_base_model_trainer.train()


In [ ]:
# Save the final model checkpoint
facebook_bart_base_model.save_pretrained('./results/FB_Bart_model')
facebook_bart_tokenizer.save_pretrained('./results/FB_Bart_model')

# Define and save the generation configuration
generation_config = GenerationConfig(
    early_stopping=True,
    num_beams=4,
    no_repeat_ngram_size=3,
    forced_bos_token_id=0,
    forced_eos_token_id=2
)

generation_config.save_pretrained('./results/FB_Bart_model')

In [ ]:
cd /content

In [ ]:
# Define paths
logs_folder = '/content/logs'  # Adjust to your source folder
logs_git_folder = './content/qM-AI-L/email-subject/model-tuning/logs'  # Adjust to your repository folder

# Copy the folder
shutil.copytree(logs_folder, logs_git_folder)

In [ ]:
# Define paths
results_folder = '/content/results'  # Adjust to your source folder
results_git_folder = './content/qM-AI-L/email-subject/model-tuning/results'  # Adjust to your repository folder

# Copy the folder
shutil.copytree(results_folder, results_git_folder)

In [ ]:
# Change directory to the cloned repository
os.chdir('./content/qM-AI-L')  # Adjust to your repository folder

# Configure Git (only needed once)
!git config --global user.email "mlreddy3082@gmail.com"
!git config --global user.name "Mlr9459"

# Add, commit, and push changes
!git add /content/qM-AI-L/email-subject/
!git commit -m "Add training outputs: model checkpoints, tokenizer, and generation configuration"
!git push origin master

**Evaluate the Model Quantitatively (with ROUGE Metric)
The ROUGE metric) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.**

In [ ]:
tokenized_train_datasets, tokenized_test_datasets

In [ ]:
tokenized_train_datasets[0:10]['cleaned_emails']

In [ ]:
emails_for_score = tokenized_test_datasets[0:10]['cleaned_emails']
human_annoted_subjects = tokenized_test_datasets[0:10]['subject']


In [ ]:
facebook_bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
facebook_bart_base_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
model_directory = '/content/results/FB_Bart_model'
facebook_bart_tokenizer_trained = BartTokenizer.from_pretrained(model_directory)
facebook_bart_base_model_trained = BartForConditionalGeneration.from_pretrained(model_directory)

In [ ]:
facebook_bart_subjects = []
trained_facebook_bart_subjects = []

for _, email in enumerate(emails_for_score):
    prompt = f"""
    Generate the subject line for the following email.

    Email:
    {email}

    Subject:
    """
    # Tokenize the input and move to the same device as the model
    inputs = facebook_bart_tokenizer(prompt, return_tensors='pt').to(facebook_bart_base_model.device)

    # Add the decoder_start_token_id to the generation configuration
    generation_config = GenerationConfig(
    max_new_tokens=200,
    decoder_start_token_id=facebook_bart_tokenizer.bos_token_id  # Add this line
    )



    # Generate subject using the original model
    original_model_outputs = facebook_bart_base_model.generate(**inputs, generation_config=generation_config)
    original_model_subject_output = facebook_bart_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    facebook_bart_subjects.append(original_model_subject_output)

    trained_facebook_bart_outputs = facebook_bart_base_model_trained.generate(**inputs, generation_config=generation_config)
    instruct_model_text_output = facebook_bart_tokenizer_trained.decode(trained_facebook_bart_outputs[0], skip_special_tokens=True)
    trained_facebook_bart_subjects.append(instruct_model_text_output)

zipped_summaries = list(zip(human_annoted_subjects, facebook_bart_subjects, trained_facebook_bart_subjects))

df = pd.DataFrame(zipped_summaries, columns = ['human_annoted_subjects', 'facebook_bart_subjects', 'trained_facebook_bart_subjects'])
df

In [ ]:
!pip install rouge-score
from rouge_score import rouge_scorer
from rouge_score.scoring import BootstrapAggregator



In [ ]:
# Calculate ROUGE scores for each pair of prediction and reference
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
aggregator = BootstrapAggregator() # Initialize aggregator

for prediction, reference in zip(facebook_bart_subjects, human_annoted_subjects[0:len(facebook_bart_subjects)]):
    score = scorer.score(reference, prediction)
    aggregator.add_scores(score) # Add scores to aggregator

# Aggregate the scores
original_model_results = aggregator.aggregate()

print(original_model_results)

In [ ]:
original_model_results = rouge.compute(
    predictions=facebook_bart_subjects,
    references=human_annoted_subjects[0:len(facebook_bart_subjects)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=trained_facebook_bart_subjects,
    references=human_annoted_subjects[0:len(trained_facebook_bart_subjects)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

In [ ]:
# Save the model locally
model_save_path = "./facebook_bart_base"
facebook_bart_base_model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

In [ ]:
# Generate subject lines for the test set
def generate_subject_line(email_body):
    inputs = tokenizer("generate subject line: " + email_body, return_tensors="pt", max_length=512, truncation=True)
    # Move inputs to the GPU if available
    inputs = {k: v.to(facebook_bart_base_model.device) for k, v in inputs.items()}
    # Access input_ids from the dictionary
    outputs = facebook_bart_base_model.generate(inputs['input_ids'], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_df['generated_subject_line'] = test_df['body'].apply(generate_subject_line)

In [ ]:
test_df

In [ ]:

import nltk
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)


# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # Tokenize the input for METEOR
    true_subject_tokens = nltk.word_tokenize(true_subject)
    generated_subject_tokens = nltk.word_tokenize(generated_subject)

    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([true_subject_tokens], generated_subject_tokens) # Pass tokenized input

    # SACREBLEU - Use import sacrebleu directly
    sacre_bleu_score = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu_score # Return the score, not the module

test_df['rougeL'], test_df['meteor'], test_df['sacrebleu'] = zip(*test_df.apply(
    lambda row: evaluate_metrics(row['subject'], row['generated_subject_line']), axis=1))

In [ ]:

# Calculate average scores
average_rougeL = test_df['rougeL'].mean()
average_meteor = test_df['meteor'].mean()
average_sacrebleu = test_df['sacrebleu'].mean() # Now you should be able to calculate the mean

print(f'Average facebook_bart_base_model ROUGE-L: {average_rougeL:.4f}')
print(f'Average facebook_bart_base_model METEOR: {average_meteor:.4f}')
print(f'Average facebook_bart_base_model SACREBLEU: {average_sacrebleu:.4f}')

In [ ]:
import shutil
from google.colab import files

# Replace 'directory_name' with the name of your directory
shutil.make_archive('/content/facebook_bart_base_model', 'zip', 'facebook_bart_base_model')

In [ ]:
# files.download('facebook_bart_base_model.zip')

# **t5-base**

In [ ]:
# Define tokenizer and model t5-base
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")


In [ ]:
dataset = Dataset.from_pandas(pivot_df)

In [ ]:
dataset

In [ ]:
def preprocess_function(pivot_df):
    inputs = pivot_df['cleaned_emails']
    targets = pivot_df['subject']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
import shutil
from google.colab import files

# Replace 'directory_name' with the name of your directory
# shutil.make_archive('/content/fine_tuned_t5', 'zip', 'fine_tuned_t5')

# Download the zipped directory


In [ ]:
shutil.make_archive('/content/results', 'zip', 'FB_Bart_model_results')
shutil.make_archive('/content/logs', 'zip', 'FB_Bart_model_logs')


In [ ]:
# files.download('fine_tuned_t5.zip')

In [ ]:
tokenized_dataset

In [ ]:
# tokenized_dataset['attention_mask']
# tokenized_dataset['input_ids']

In [ ]:
# Split the dataset into train and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Data collator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    # predict_with_generate=True,
    logging_dir="./logs",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

In [ ]:
model.save_pretrained('fine_tuned_t5')
tokenizer.save_pretrained('fine_tuned_t5')

In [ ]:
# Checking if GPU is available
if torch.cuda.is_available():
    print("GPU is available. \nUsing GPU")
    device = torch.device('cuda')
else:
    print("GPU is not available. \nUsing CPU")
    device = torch.device('cpu')

In [ ]:
def generate_summary(text):
    inputs = tokenizer.encode_plus(text, return_tensors='pt', max_length=512, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    summary_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=150, num_beams=2, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary



In [ ]:
# Example inference
example_text = pivot_df['body'][3]
print("Email Body:", example_text)
print("\n\n Generated Subject:", generate_summary(example_text))
print("\n\n Actual Subject:", pivot_df['subject'][3])

In [ ]:
from rouge_score import rouge_scorer
import sacrebleu
from nltk.translate.meteor_score import meteor_score

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Evaluate the generated subject lines
def evaluate_metrics(true_subject, generated_subject):
    # ROUGE-L
    rougeL = scorer.score(true_subject, generated_subject)['rougeL'].fmeasure

    # METEOR
    meteor = meteor_score([true_subject], generated_subject)

    # SACREBLEU
    sacre_bleu = sacrebleu.corpus_bleu([generated_subject], [[true_subject]]).score

    return rougeL, meteor, sacre_bleu

test_df['rougeL'], test_df['meteor'], test_df['sacrebleu'] = zip(*test_df.apply(
    lambda row: evaluate_metrics(row['subject_line'], row['generated_subject_line']), axis=1))

# Calculate average scores
average_rougeL = test_df['rougeL'].mean()
average_meteor = test_df['meteor'].mean()
average_sacrebleu = test_df['sacrebleu'].mean()

print(f'Average ROUGE-L: {average_rougeL:.4f}')
print(f'Average METEOR: {average_meteor:.4f}')
print(f'Average SACREBLEU: {average_sacrebleu:.4f}')

In [ ]:
import re

def extract_sentences(text):
  """
  This function extracts sentences from text without introducing control characters.

  Args:
      text (str): The text to be processed.

  Returns:
      list: A list of extracted sentences.
  """
  # Split text based on sentence boundaries (adjust pattern as needed)
  sentences = re.split(r"(?<!\w)\.(?!\w)", text)

  # Remove empty elements and leading/trailing whitespace
  sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

  return sentences

# Example usage
text = "This is an example text. It contains multiple sentences.  \nThere are also newlines and extra spaces."
extracted_sentences = extract_sentences(pivot_df['body'][0])
print(extracted_sentences)


In [ ]:
pivot_df['body'][0]

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summary = summarizer(pivot_df['body'][0], max_length=15, min_length=2, do_sample=False)
print(summary[0]['summary_text'])

In [ ]:
pivot_df['body'][0]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("Chirayu/subject-generator-t5-base")
tokenizer = AutoTokenizer.from_pretrained("Chirayu/subject-generator-t5-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def get_subject(content, num_beams=5,max_length=512, repetition_penalty=2.5, length_penalty=1, early_stopping=True,top_p=.95, top_k=50, num_return_sequences=3):

  text =  "title: " + content + " </s>"

  input_ids = tokenizer.encode(
    text, return_tensors="pt", add_special_tokens=True
  )

  input_ids = input_ids.to(device)
  generated_ids = model.generate(
      input_ids=input_ids,

      num_beams=num_beams,
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      length_penalty=length_penalty,
      early_stopping=early_stopping,
      top_p=top_p,
      top_k=top_k,
      num_return_sequences=num_return_sequences,
  )
  subjects = [tokenizer.decode(generated_id,skip_special_tokens=True,clean_up_tokenization_spaces=True,) for generated_id in generated_ids]
  return subjects

In [ ]:
x = get_subject(pivot_df['body'][0])
x

In [ ]:
pivot_df['body'][0]

In [ ]:
pivot_df['body'][1]

In [ ]:
import textwrap

In [ ]:
def text_summarizer(email_text):

    model_name = "facebook/bart-large-cnn"
    model = BartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    inputs = tokenizer.encode("summarize: " + email_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=100, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    formatted_summary = "\n".join(textwrap.wrap(summary, width=80))
    return formatted_summary

summary = text_summarizer(pivot_df['body'][0])
summary

In [ ]:
def extract_sentences(text):
    """
    Extract sentences from the text and remove newline characters.

    Parameters:
    text (str): The text to process.

    Returns:
    list: A list of sentences without newline characters.
    """
    # Replace newline characters with spaces
    text = text.replace('\n', '')
    text = text.replace('!', '.').replace('?', '.').replace(';', '.')

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    return sentences

# Example usage
sample_text = "Hello, world!\nThis is a test.\nLet's replace, commas, and newlines."
sentences = extract_sentences(sample_text)
for sentence in sentences:
    print(sentence)

In [ ]:
summary = text_summarizer(pivot_df['body'][0])
summary

In [ ]:
pivot_df['body'][0]

In [ ]:
email_len = [len(pivot_df['dialogue'].split()) for x in samsum['train']]
sub_len = [len(pivot_df['summary'].split()) for x in samsum['train']]

dialogue_len = [len(x['dialogue'].split()) for x in samsum['train']]
summary_len = [len(x['summary'].split()) for x in samsum['train']]



In [ ]:
describe_df(pivot_df)

In [ ]:
nan_rows

In [ ]:
from langchain.preprocessing import Tokenizer, Lowercaser, PunctuationRemover, StopwordRemover
from langchain.stemming import Stemmer
from langchain import Pipeline

In [ ]:
len(email_docs)*3

In [ ]:
emails_df['content'][0]

In [ ]:
emails_df['source'][0]

In [ ]:
extract_file_name(emails_df['source'][0])